In [1]:
using Distributed

using Pkg


@everywhere begin 
    using ParallelTemperingMonteCarlo
    using DelimitedFiles
    using BenchmarkTools
end

In [9]:
using Random
@everywhere import ..MCRun: initialise_histograms!,updatehistogram!,update_max_stepsize!,sampling_step!,save_results,save_states
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 50.
tf = 160.
n_traj = 16

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 1000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
 [0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
 [0.1811648830,        0.5575692066,        0.2931316748],
 [0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
 [0.4742970144,        0.3445967202,       -0.2931309590],
 [0.4742970205,       -0.3445967231,       -0.2931309559],
 [0.0000007214,       -0.0000000073,       -0.6554619185]]

#convert to Bohr
nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu13 = copperconstant*ico_13*1.5
AtoBohr = 1.8897259886


#boundary conditions 
bc_cu13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_cu13, bc_cu13)

#histogram information
n_bin = 100




100

In [10]:
 runnerdir = "/home/ghun245/RuNNer-testing/Brass_potential/"
 atomtype="Cu"
 pot = AbstractMLPotential(runnerdir,atomtype)
# RuNNer.getRuNNerenergy(runnerdir,1;input_idx=2)

AbstractMLPotential("/home/ghun245/RuNNer-testing/Brass_potential/", "Cu")

In [11]:
pot_vector = []

for i =1:Threads.nthreads()
    potential=ParallelMLPotential(runnerdir,"Cu",i)
    push!(pot_vector,potential)
end

mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot_vector[1] ; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]
    

results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

In [5]:
parallel_states,a,v,r,delta_en,n_threads = parallel_equilibration(mc_states,move_strat,mc_params,pot_vector[1],ensemble,results)

(Any[MCState{Float64, 13, SphericalBC{Float64}}[MCState{Float64, 13, SphericalBC{Float64}}(50.0, 6315.499705261945, Config{13, SphericalBC{Float64}, Float64}(StaticArraysCore.SVector{3, Float64}[[-0.09850579885821831, -0.005157842443408889, 0.1248460009153229], [0.13383528664474098, 0.003456137642791279, 7.04811993740721], [2.0184164854865636, -5.796311048825306, 3.2924251793792787], [-4.874670455235371, -3.541653030973376, 3.01270453061964], [-4.907591293403073, 3.7007720913548483, 3.0607727753581417], [1.8619537075439925, 5.730514845094081, 3.0127119541839686], [6.316424044033252, -0.16341367787579042, 3.1426256436771247], [-1.868257199897207, -6.026474706790241, -2.9984610601378052], [-6.323036830175243, 0.11495671003974081, -3.103324778832583], [-1.8645412856453434, 5.875118034708346, -3.062523047135528], [5.0988286730956505, 3.6457034703958944, -3.2180436114954114], [4.977810402656452, -3.6425922295152917, -3.0897871786350253], [-0.146681160069665, 0.011217527401057414, -6.9726792

In [6]:
function testparallelrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    Threads.@threads for i=1:4
        for j=1:1000
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

function testrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    for i=1:4
        for j=1:1000
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

testrunner (generic function with 1 method)

In [12]:
ico_55 = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]
#convert to Bohr
nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu55 = copperconstant*ico_55
pos_cu13 = copperconstant*ico_13*1.5
AtoBohr = 1.8897259886

# length(pos_cu55) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_cu55 = SphericalBC(radius=14*AtoBohr)   #5.32 Angstrom
bc_cu13 = SphericalBC(radius=8*AtoBohr)
#starting configuration
start_config = Config(pos_cu55, bc_cu55)
#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot_vector[1]; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]
mc_cycles = 50 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

MCParams(50, 50, 1, 16, 13, 100)

In [13]:
parallel_states,pot_vec,a,v,r,delta_en,n_threads = parallel_equilibration(mc_states,move_strat,mc_params,pot_vector[1],ensemble,results)

(Any[MCState{Float64, 55, SphericalBC{Float64}}[MCState{Float64, 55, SphericalBC{Float64}}(50.0, 6315.499705261945, Config{55, SphericalBC{Float64}, Float64}(StaticArraysCore.SVector{3, Float64}[[-0.07854356285398179, -0.06822737498855043, 0.05897410689038068], [-3.980885942754e-6, -3.3375035158614e-6, 4.575916827038472], [1.338113226475645, -3.8896639952661753, 2.1265962899424724], [-3.311163519470279, -2.405710469814843, 2.0464114164455425], [-3.300195429377905, 2.342766914179879, 2.1345570684062736], [1.264752859461583, 3.8924983184634674, 2.046406418069819], [4.092824837722801, -8.6887460654154e-6, 2.046420003785219], [-1.2647438377188622, -3.8925245957365804, -2.046405726039733], [-4.092815815294902, -1.75878418699746e-5, -2.0464193117551326], [-1.2911076960990706, 3.8891604194743388, -2.1119511537741436]  …  [2.0744963372369405, 6.384583971948036, -4.148977939714466], [6.5594487104586, 4.765688045720408, -4.053952548518178], [6.713184197946128, -2.1916113212552402e-5, -4.14895565

In [9]:
function testparallelrunner2(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    Threads.@threads for i=1:8
        for j=1:10
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

function testrunner2(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    for i=1:8
        for j=1:10
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

testrunner2 (generic function with 1 method)

In [15]:
@benchmark testrunner2(parallel_states,a,v,r,delta_en,n_threads,pot_vector)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 88.506 s (0.01% GC) to evaluate,
 with a memory estimate of 151.65 MiB, over 1892130 allocations.

In [16]:
@benchmark testparallelrunner2(parallel_states,a,v,r,delta_en,n_threads,pot_vector)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 23.860 s (0.10% GC) to evaluate,
 with a memory estimate of 171.28 MiB, over 2669923 allocations.

In [14]:
@benchmark ParallelRun.pptmc_cycle(parallel_states,results,move_strat,pot_vector,n_threads,delta_en)

TaskFailedException: TaskFailedException

    nested task error: UndefVarError: mc_params not defined
    Stacktrace:
     [1] macro expansion
       @ ~/ParallelTemperingMonteCarlo.jl/src/parallelrun.jl:163 [inlined]
     [2] (::ParallelTemperingMonteCarlo.ParallelRun.var"#106#threadsfor_fun#4"{ParallelTemperingMonteCarlo.ParallelRun.var"#106#threadsfor_fun#3#5"{Vector{Any}, Output{Float64}, MoveStrategy{13, 0, 0}, Vector{Any}, Float64, UnitRange{Int64}}})(tid::Int64; onethread::Bool)
       @ ParallelTemperingMonteCarlo.ParallelRun ./threadingconstructs.jl:84
     [3] #106#threadsfor_fun
       @ ./threadingconstructs.jl:51 [inlined]
     [4] (::Base.Threads.var"#1#2"{ParallelTemperingMonteCarlo.ParallelRun.var"#106#threadsfor_fun#4"{ParallelTemperingMonteCarlo.ParallelRun.var"#106#threadsfor_fun#3#5"{Vector{Any}, Output{Float64}, MoveStrategy{13, 0, 0}, Vector{Any}, Float64, UnitRange{Int64}}}, Int64})()
       @ Base.Threads ./threadingconstructs.jl:30

In [19]:
input_idx = 1
run(`./RuNNer$input_idx.x $input_idx `)

Process(`./RuNNer1.x 1`, ProcessExited(0))